## Cencepts from paper

"Measuring Semantic Similarity between Concepts in Visual Domain" Table 1

In [1]:
concepts = [
    "bay",
    "beach",
    "birds",
    "boeing",
    "buildings",
    "city",
    "clouds",
    "face",
    "f-16",
    "helicopter",
    "mountain",
    "sky",
    "ships",
    "sunset",
    "sunrise",
    "ocean"
]

In [2]:
len(concepts)

16

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
chrome_options = Options()
chrome_options.binary_location = '/usr/bin/google-chrome-stable'
chrome_options.add_argument("--headless")
#chrome_options.add_argument('--disable-gpu')

# fix for chrome not reachable
#chrome_options.add_argument('--no-sandbox')

In [3]:
driver = webdriver.Chrome(chrome_options=chrome_options, executable_path="/usr/local/bin/chromedriver")
driver.maximize_window()

In [4]:
# selenium test.

# driver.get('http://www.google.com')
# print(driver.title)
# driver.close()

In [5]:
top_url = "http://images.search.yahoo.com/"

In [6]:
def search_word(word):
    driver.get(top_url)
    input = driver.find_element_by_id('yschsp')
    input.send_keys(word)
    btn = driver.find_element_by_class_name('ygbt')
    btn.click()

In [7]:
search_word('bay')

In [8]:
driver.title

'bay - Yahoo Image Search Results'

In [9]:
first_res = driver.find_element_by_id("resitem-0")

In [10]:
par = first_res.parent

In [11]:
all_lis = par.find_elements_by_tag_name('li')

In [12]:
import time
def ctrl_click_to_newtab(link):
    for retry in range(5):
        ActionChains(driver).key_down(Keys.CONTROL).click(link).key_up(Keys.CONTROL).perform()
        if len(driver.window_handles) == 2:
            return
        time.sleep(0.5)
    raise Exception("do not open new tab")


In [13]:
def get_image_url_from_viewimage():
    link = driver.find_element_by_link_text('View Image')
    ctrl_click_to_newtab(link)
    driver.switch_to_window(driver.window_handles[1])
    time.sleep(1)
    url = driver.current_url
    driver.close()
    time.sleep(0.5)
    driver.switch_to_window(driver.window_handles[0])
    time.sleep(0.5)
    return url

In [14]:
def get_image_url_from_iholder():
    div = driver.find_element_by_class_name('iholder')
    img = div.find_element_by_id('img')
    return img.get_attribute('src')

In [15]:
def get_image_url(image_li):
    image_li.click()
    time.sleep(0.5)
    return get_image_url_from_viewimage()

In [16]:
def click_more():
    more = driver.find_element_by_name('more-res')
    more.click()

In [17]:
def find_index_by_id(all_li, id):
    for i in range(len(all_li)):
        if(all_li[i].get_attribute('id') == id):
            return i
    return -1

In [18]:
def click_next():
    driver.find_element_by_css_selector(".nav.right").click()

In [19]:
from selenium.common.exceptions import StaleElementReferenceException, WebDriverException
def get_all_url(all_li, from_count=0, limit=20):
    res = []
    lastid = None
    begin = find_index_by_id(all_li, "resitem-{}".format(from_count))
    all_li[begin].click()
    time.sleep(0.5)
    count = from_count
    while count < limit:
        try:
            print(count)
            res.append(get_image_url_from_iholder())
        except WebDriverException:
            print("WebDriverException: Please restart from here.")
            return(count, res)
        click_next()
        time.sleep(1)
        count +=1
    return (count, res)

In [22]:
count, urls = get_all_url(all_lis, limit=100)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
WebDriverException: Please restart from here.


In [22]:
count, urls = get_all_url(all_lis, limit=100)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
WebDriverException: Please restart from here.


In [23]:
urls

['https://tse1.mm.bing.net/th?id=OIP.5Su3_66BqHzeAhUAbUgklQEsDI&pid=15.1&P=0&w=259&h=174',
 'http://upload.wikimedia.org/wikipedia/commons/9/9d/Smitswinkel_Bay.jpg',
 'http://www.worldfortravel.com/wp-content/uploads/2013/04/Freycinet-Australia.jpg',
 'http://ciaobambino.com/wp-content/uploads/2013/08/little-dix-bay-beach.jpg',
 'http://www.worldofwanderlust.com/wp-content/uploads/2015/07/P1010074.jpg',
 'https://media-cdn.tripadvisor.com/media/photo-s/01/c5/d9/3f/view-of-the-bay-from.jpg',
 'http://mustseeplaces.eu/wp-content/uploads/2016/01/freycinet-national-park-tasmania-australia.jpg',
 'http://upload.wikimedia.org/wikipedia/commons/6/6f/St_oswalds_bay_near_lulworth_dorset_arp.jpg',
 'http://upload.wikimedia.org/wikipedia/commons/c/ce/Wineglass_Bay_from_Lookout_crop.jpg',
 'https://www.engelvoelkers.com/wp-content/uploads/2015/05/Clearwater_Bay.jpg',
 'http://static.thousandwonders.net/Hanauma.Bay.original.15812.jpg',
 'http://internet-travelgroups.com/wp-content/uploads/2016/01/W

In [23]:
count

21

In [22]:
count, urls = get_all_url(all_lis, from_count=21, limit=100)

0
1
2
3
4
5
6
WebDriverException: Please restart from here.


In [22]:
count, urls = get_all_url(all_lis, from_count=22, limit=100)

22
23
24
25
26
27
WebDriverException: Please restart from here.


In [23]:
count

27

In [24]:
urls

['https://tse3.mm.bing.net/th?id=OIP.QZT2x02R51ZdgjFCwnY6BwEgDY&pid=15.1&P=0&w=220&h=165',
 'http://askbeach.com/wp-content/uploads/2016/01/wineglass_6.jpg',
 'http://www.pgl.co.uk/Files/Files/Schools/Primary%20Schools/Centre%20Gallery/Osmington%20Bay/PS-G-Centre-Osmington-Bay-sea-beach-durdle-door-jurassic-coast-for-Primary-Schools.jpg',
 'https://tse4.mm.bing.net/th?id=OIP._ps0dCwgM-_IO4R9q-jC3wEsC7&pid=15.1&P=0&w=274&h=172',
 'http://www.100besteverything.com/best-beaches/i-53-1924_hanaumabaywp1.jpg']

In [22]:
count, urls = get_all_url(all_lis, from_count=27, limit=100)

27
28
WebDriverException: Please restart from here.


In [23]:
urls

['https://tse3.mm.bing.net/th?id=OIP.wcqIe65uyBMM1hW40t2QpQEsDI&pid=15.1&P=0&w=257&h=173']

In [20]:
count, urls = get_all_url(all_lis, from_count=28, limit=100)

28
29
30
31
32
WebDriverException: Please restart from here.


In [21]:
urls

['https://tse3.mm.bing.net/th?id=OIP.yINJMexEXGs7QoDx6KsGzgEsDo&pid=15.1&P=0&w=220&h=171',
 'http://www.touristsbook.com/wp-content/uploads/2016/01/wineglass-bay-tasmnia.jpg',
 'http://1.bp.blogspot.com/-mfYgwpoi5-0/UOQ98ZrCJcI/AAAAAAAAAts/Y1pXc0pMcAY/s1600/Byron+Bay+View.jpg',
 'https://www.cornwall-beaches.co.uk/public/photos/lantic-bay-beach.jpg']

In [20]:
count, urls = get_all_url(all_lis, from_count=32, limit=100)

32
33
34
35
36
WebDriverException: Please restart from here.


In [21]:
urls

['https://tse3.mm.bing.net/th?id=OIP.8YpLRoCzCJNYR3eC0jTd5wEsDI&pid=15.1&P=0&w=246&h=165',
 'https://tse4.mm.bing.net/th?id=OIP.uyrbgx3_iVSAMugF3F1tVgE1DK&pid=15.1&P=0&w=319&h=209',
 'http://images.trvl-media.com/media/content/shared/images/travelguides/Byron-Bay-and-vicinity-6138873-smallTabletRetina.jpg',
 'http://www.doc.govt.nz/global/images/places/northland/maitai-bay/maitai-bay-karikari-peninsula-gallery.jpg']

In [20]:
count, urls = get_all_url(all_lis, from_count=36, limit=100)

36
37
38
WebDriverException: Please restart from here.


In [21]:
urls

['https://tse3.mm.bing.net/th?id=OIP.TCQAnyFjNmo-37TVobOX1gEsB1&pid=15.1&P=0&w=368&h=144',
 'http://www.carlisle-bay.com/images/carlisle-bay-hotel_antigua-compressed.jpg']

In [20]:
count, urls = get_all_url(all_lis, from_count=38, limit=100)

38
39
WebDriverException: Please restart from here.


In [21]:
urls

['https://tse4.mm.bing.net/th?id=OIP.ijApGq_tsLGpakNyR7X9GwEsBd&pid=15.1&P=0&w=368&h=115']

In [20]:
count, urls = get_all_url(all_lis, from_count=39, limit=100)

39
40
41
42
43
44
45
46
WebDriverException: Please restart from here.


In [22]:
urls

['https://tse1.mm.bing.net/th?id=OIP.QWVv30E5K-M_78aWQvlaWQEsDI&pid=15.1&P=0&w=256&h=172',
 'https://www.leisuretours.co.nz/wp-content/uploads/Bay-of-Islands.jpg',
 'https://upload.wikimedia.org/wikipedia/commons/thumb/4/42/Pillar_Point_and_Half_Moon_Bay.jpg/1200px-Pillar_Point_and_Half_Moon_Bay.jpg',
 'https://upload.wikimedia.org/wikipedia/commons/d/d2/Clear_Water_Bay_1st_Beach.jpg',
 'https://media-cdn.tripadvisor.com/media/photo-o/0d/d2/f0/3a/plantation-bay-resort.jpg',
 'https://d19lgisewk9l6l.cloudfront.net/wexas/www/images/largeimages/destinations/Antigua-and-Barbuda/Carlisle_Bay.jpg',
 'http://askbeach.com/wp-content/uploads/2016/01/horseshoe_2.jpg']

In [19]:
get_image_url(all_lis[106])

'https://www.iod.org.nz/portals/0/Branches%20and%20events/BayofPlenty_header.jpg'

In [20]:
get_image_url(all_lis[107])

'http://www.tourstas.com.au/wp-content/uploads/2014/05/Wineglass_Bay_2_940x350.jpg'

In [21]:
get_image_url(all_lis[108])

'http://tassiegrammer.com.au/wp-content/uploads/2016/01/WineglassBay3.jpg'

In [22]:
get_image_url(all_lis[109])

WebDriverException: Message: unknown error: session deleted because of page crash
from tab crashed
  (Session info: headless chrome=62.0.3202.75)
  (Driver info: chromedriver=2.33.506092 (733a02544d189eeb751fe0d7ddca79a0ee28cce4),platform=Linux 4.9.0-4-amd64 x86_64)


In [ ]:
get_image_url(all_lis[110])

In [14]:
list(map(lambda el: el.get_attribute("id"), all_lis))

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'pivot_more',
 '',
 '',
 '',
 '',
 '',
 '',
 'yss',
 '',
 '',
 'license',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'resitem-fpub0',
 'resitem-fpub1',
 'resitem-fpub2',
 'resitem-fpub3',
 'resitem-fpub4',
 'resitem-fpub5',
 'resitem-fpub6',
 'resitem-fpub7',
 'resitem-fpub8',
 'resitem-fpub9',
 'resitem-fpub10',
 'resitem-fpub11',
 'resitem-fpub12',
 'resitem-fpub13',
 'resitem-fpub14',
 'resitem-fpub15',
 'resitem-fpub16',
 'resitem-fpub17',
 'resitem-fpub18',
 'resitem-fpub19',
 '',
 'resitem-0',
 'resitem-1',
 'resitem-2',
 'resitem-3',
 'resitem-4',
 'resitem-5',
 'resitem-6',
 'resitem-7',
 'resitem-8',
 'resitem-9',
 'resitem-10',
 'resitem-11',
 'resitem-12',
 'resitem-13',
 'resitem-14',
 'resitem-15',
 'resitem-16',
 'resitem-17',
 'resitem-

In [16]:
first_res.click()

In [17]:
driver.find_element_by_class_name('iholder')

<selenium.webdriver.remote.webelement.WebElement (session="48c95370da3fbfad83f4ade1687ce9f0", element="0.5663800474746361-160")>

In [18]:
div = driver.find_element_by_class_name('iholder')

In [19]:
div.find_element_by_id('img')

<selenium.webdriver.remote.webelement.WebElement (session="48c95370da3fbfad83f4ade1687ce9f0", element="0.5663800474746361-161")>

In [20]:
img = div.find_element_by_id('img')

In [21]:
img.get_attribute('src')

'https://achillfieldschool.files.wordpress.com/2014/05/dsc_0287.jpg'

In [22]:
get_image_url_from_viewimage()

'https://achillfieldschool.files.wordpress.com/2014/05/dsc_0287.jpg'

In [29]:
click_next()

In [30]:
get_image_url_from_iholder()

'http://www.worldfortravel.com/wp-content/uploads/2013/04/Freycinet-Australia.jpg'

In [24]:
(20%10) == 0

True

In [20]:
all_lis[88].get_attribute('id')

'resitem-0'

In [26]:
get_image_url(all_lis[88])

'https://achillfieldschool.files.wordpress.com/2014/05/dsc_0287.jpg'

In [27]:
get_image_url(all_lis[89])

'https://upload.wikimedia.org/wikipedia/commons/9/9d/Smitswinkel_Bay.jpg'

In [29]:
get_image_url(all_lis[90]), get_image_url(all_lis[91]), get_image_url(all_lis[92]), get_image_url(all_lis[93])

('http://www.worldfortravel.com/wp-content/uploads/2013/04/Freycinet-Australia.jpg',
 'http://www.worldfortravel.com/wp-content/uploads/2013/04/Freycinet-Australia.jpg',
 'http://www.worldofwanderlust.com/wp-content/uploads/2015/07/P1010074.jpg',
 'https://media-cdn.tripadvisor.com/media/photo-s/01/c5/d9/3f/view-of-the-bay-from.jpg')

In [25]:
all_lis[140].get_attribute('id')

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: headless chrome=62.0.3202.75)
  (Driver info: chromedriver=2.33.506092 (733a02544d189eeb751fe0d7ddca79a0ee28cce4),platform=Linux 4.9.0-4-amd64 x86_64)


In [38]:
all_lis[89].click()

In [23]:
link = driver.find_element_by_link_text('View Image')

In [24]:
link

<selenium.webdriver.remote.webelement.WebElement (session="4a6d7e84a7ea8ce3d878bd29253b6b63", element="0.34873234290672506-160")>

In [20]:
all_lis[92].click()

In [21]:
link = driver.find_element_by_link_text('View Image')
ActionChains(driver).key_down(Keys.CONTROL).click(link).key_up(Keys.CONTROL).perform()

In [40]:
driver.window_handles

WebDriverException: Message: no such session
  (Driver info: chromedriver=2.33.506092 (733a02544d189eeb751fe0d7ddca79a0ee28cce4),platform=Linux 4.9.0-4-amd64 x86_64)


In [120]:
all_lis2[143].get_attribute("id").startswith('resitem')

True

In [129]:
all_lis2[93].get_attribute('innerHTML')

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: headless chrome=62.0.3202.75)
  (Driver info: chromedriver=2.33.506092 (733a02544d189eeb751fe0d7ddca79a0ee28cce4),platform=Linux 4.9.0-4-amd64 x86_64)


In [31]:
first_res.click()

In [109]:
find_index_by_id(all_lis2, 'resitem-49')

139

In [125]:
find_index_by_id(all_lis2, 'resitem-0')

88

In [ ]:
#from time import sleep
#sleep(0.05)

In [68]:
def find_index(list, target):
    for i in range(len(list)):
        if(list[i].id == target.id):
            return i
    return -1

In [69]:
find_index(all_lis, first_res)

88

In [83]:
len(all_lis)

159

In [71]:
all_lis[88].get_attribute("id")

'resitem-0'

In [74]:
all_lis[141].get_attribute("id")

'resitem-53'

In [75]:
all_lis[142].get_attribute("id")

'yui_3_5_1_1_1509360357945_752'

In [87]:
all_lis[150].get_attribute("id")

'yui_3_5_1_1_1509360357945_784'

In [33]:
link = driver.find_element_by_link_text('View Image')

In [37]:
ActionChains(driver).key_down(Keys.CONTROL).click(link).key_up(Keys.CONTROL).perform()

In [38]:
driver.window_handles

['CDwindow-e0a609a0-3c4e-48fa-9d75-3e428a57205f',
 'CDwindow-0e5a14ab-ae1b-4ce7-8b61-5771e4070e18']

In [42]:
driver.current_window_handle

'CDwindow-e0a609a0-3c4e-48fa-9d75-3e428a57205f'

In [44]:
driver.switch_to_window(driver.window_handles[1])

In [45]:
driver.current_url

'https://upload.wikimedia.org/wikipedia/commons/9/9d/Smitswinkel_Bay.jpg'

In [46]:
urllist = []

In [47]:
urllist.append(driver.current_url)

In [48]:
driver.close()

In [49]:
driver.window_handles

['CDwindow-e0a609a0-3c4e-48fa-9d75-3e428a57205f']

In [51]:
driver.switch_to_window(driver.window_handles[0])

In [52]:
driver.current_url

'http://images.search.yahoo.com/search/images;_ylt=AwrTcXiu9fZZ1JcAkweLuLkF;_ylc=X1MDOTYwNTc0ODMEX3IDMgRiY2sDZTZycGdwNWN2ZHRkZSUyNmIlM0QzJTI2cyUzRHEyBGZyAwRncHJpZANSMWd6aFl0eFE5dUtKbk1KM2V6dVJBBG10ZXN0aWQDbnVsbARuX3N1Z2cDMTAEb3JpZ2luA2ltYWdlcy5zZWFyY2gueWFob28uY29tBHBvcwMwBHBxc3RyAwRwcXN0cmwDBHFzdHJsAzMEcXVlcnkDYmF5BHRfc3RtcAMxNTA5MzU2OTc4BHZ0ZXN0aWQDbnVsbA--?gprid=R1gzhYtxQ9uKJnMJ3ezuRA&pvid=A1d10TEwLjJUhulTd3DH.AAyMTA0LgAAAACOSmPr&fr2=sb-top-images.search.yahoo.com&p=bay&ei=UTF-8&iscqry=&fr=sfp#id=0&iurl=http%3A%2F%2Fupload.wikimedia.org%2Fwikipedia%2Fcommons%2F9%2F9d%2FSmitswinkel_Bay.jpg&action=click'

In [77]:
get_image_url(first_res)

'https://achillfieldschool.files.wordpress.com/2014/05/dsc_0287.jpg'

In [78]:
driver.window_handles

['CDwindow-46fad65f-8a9e-4fcd-9bb0-2e859f5bd4c5']

In [79]:
get_image_url(all_lis[89])

'https://upload.wikimedia.org/wikipedia/commons/9/9d/Smitswinkel_Bay.jpg'

In [80]:
get_image_url(all_lis[90])

'http://www.worldfortravel.com/wp-content/uploads/2013/04/Freycinet-Australia.jpg'

In [81]:
driver.window_handles

['CDwindow-46fad65f-8a9e-4fcd-9bb0-2e859f5bd4c5']

In [115]:
lis = [1, 2, 3]

In [116]:
lis.extend([4, 5, 6])

In [117]:
lis

[1, 2, 3, 4, 5, 6]

In [92]:
all_lis2 = par.find_elements_by_tag_name('li')

In [93]:
len(all_lis2)

199

In [99]:
all_lis2[145].get_attribute("id")

'resitem-58'

In [ ]:
driver.close()

In [ ]:
"""
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("disable-infobars")
chrome_options.add_argument("--disable-extensions")
"""